In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy
import skimage
# import custom functions
import sys
sys.path.append("../../../../")
import utils
import os
from tqdm import tqdm
import torch
from skimage.io import imread
from scipy.io import loadmat
from skimage.measure import regionprops
from skimage.transform import resize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re
import graph

from skimage.transform import resize
from skimage.measure import regionprops
from scipy.io import loadmat
from skimage.io import imread


In [4]:
# load data

load_path = '/mnt/cloud1/sheng-projects/st_projects/spatial_clust/data/tonsil/'
metaload_path = '/mnt/cloud1/sheng-projects/st_projects/spatial_clust/data/tonsil/processed_data/meta_data/'
df_clean = pd.read_csv(os.path.join(metaload_path , "features_and_metadata.csv"), index_col=0) # already cleaned
xrange = [ 4, 5, 6, 7, 8]
yrange = [8, 9, 10, 11, 12, 13]
df_clean = df_clean[df_clean["X_view"].isin(xrange) & df_clean["Y_view"].isin(yrange)]
df_clean.shape

(102574, 55)

In [8]:
data_dir = '/mnt/cloud1/sheng-projects/st_projects/spatial_clust/spatial-clust-scripts/ipynb/Bokai_reorg/tonsil/data/'
df = df_clean

In [9]:
# save out
# feature edge
# feature label
# cell nbhd
# spatial edge

features_names = [
        'HOECHST1', 'CD38', 'CD19', 'CD31',
        'Vimentin', 'CD22', 'Ki67', 'CD8', 'CD90', 'CD123', 'CD15', 'CD3',
        'CD152', 'CD21', 'cytokeratin', 'CD2', 'CD66', 'collagen IV', 'CD81',
        'HLA-DR', 'CD57', 'CD4', 'CD7', 'CD278', 'podoplanin', 'CD45RA', 'CD34',
        'CD54', 'CD9', 'IGM', 'CD117', 'CD56', 'CD279', 'CD45', 'CD49f', 'CD5',
        'CD16', 'CD63', 'CD11b', 'CD1c', 'CD40', 'CD274', 'CD27', 'CD104',
        'CD273', 'FAPalpha', 'Ecadherin'
    ]
features = df[features_names].to_numpy()
features = utils.center_scale(features)

res = 0.5
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_edges = graph.get_spatial_edges(
    arr=locations, n_neighbors=15, verbose=True
)
np.save(os.path.join(data_dir, f'spatial_edges_0325.npy'), np.array(spatial_edges[:2]).T)

/home/bokai/miniconda3/envs/cellsnap/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculating cell neighborhood composition matrix...


In [10]:
## just check dims

print([cell_nbhd.shape, len(np.unique(feature_labels))])

[(102574, 12), 12]


In [11]:
# in case need use
np.save(os.path.join(data_dir, f'feature_scaled.npy'), features)

Below is custom code to process CODEX tonsil images.

First we need to again piece the seperated FOVs into the real tissue tile.

Then produce cropped images out (save for CNN process).

In [ ]:
## custom code written for this tonsil dataset, to piece images etc

def save_train_test_data(df, size, load_path, save_folder, xrange,yrange, ratio, truncate = 3000, shape_of_each_view=(1008, 1344),
                               channels = ('CD45', 'HOECHST1'), shape_of_views=(11, 9), pad = 500, verbose = True):
    '''
    For all cells in the dataframe, locate it in the picture and crop the image of given size, padding 0 if needed
    Only select channels in the argument
    Save numpy array of size n_cells * size * size * n_channels
    
    
    Args:
    df is a pandas dataframe consisting of clean cells
    '''
    df["X_view"] = df["PointNum"].apply(lambda x: int(x.split('_')[1][1:]))
    df["Y_view"] = df["PointNum"].apply(lambda x: int(x.split('_')[2][1:]))
    df = df[df["X_view"].isin(xrange) & df["Y_view"].isin(yrange)]
    n_cell = df.shape[0]
    print(f"We have {n_cell} cells in total")
    n_channel = len(channels)
    if verbose:
        print("Combining all views...", flush = True)
    ### make all view into one big view

    
    all_folders = os.listdir(os.path.join(load_path, 'processed_raw_data/Images_singleChannel/'))
    channels = ('CD45', 'HOECHST1')
    pad_image = np.zeros((11*1008, 9*1344, 2))
    for x in range(2, 11):
        for y in range(5, 16):
            # get folder name
            x_str, y_str = ('0' + str(x))[-2:], ('0' + str(y))[-2:]
            curr_folder = [name for name in all_folders if re.search('X' + x_str + '_Y' + y_str, name)]
            assert len(curr_folder) == 1
            curr_folder = curr_folder[0]
            all_img_names = os.listdir(os.path.join(load_path, 'processed_raw_data/Images_singleChannel', curr_folder))
            for channel_id, channel in enumerate(channels):
                img_filename = [name for name in all_img_names if re.search(channel+'\W', name)]
                assert len(img_filename) == 1
                img_filename = img_filename[0]
                img = imread(
                        os.path.join(
                            load_path, 'processed_raw_data/Images_singleChannel', curr_folder, img_filename
                        )
                    )
                pad_image[(y-5)*1008:(y-4)*1008, (x-2)*1344:(x-1)*1344, channel_id] = img
    zero_pad_entire_view = np.zeros((pad_image.shape[0]+2*pad, pad_image.shape[1]+2*pad, pad_image.shape[2]))
    zero_pad_entire_view[pad:pad_image.shape[0]+pad, pad:pad_image.shape[1]+pad, :] = pad_image

#     zero_pad_entire_view[zero_pad_entire_view <= truncate] = 0
#     zero_pad_entire_view[zero_pad_entire_view > truncate] = 1
    if verbose:
        print("Processing each cell...and saving!", flush = True)
    for i in tqdm(range(n_cell)):
        # process each cell
        center_x = df.iloc[i]["centroid_x"]
        center_y = df.iloc[i]["centroid_y"]
        temp_image = resize(zero_pad_entire_view[(int(center_x-size*ratio/2)+pad):(int(center_x+size*ratio/2)+pad), 
                                               (int(center_y-size*ratio/2)+pad):(int(center_y+size*ratio/2)+pad), 
                                               :], (size, size))
        cur_image = np.zeros(temp_image.shape)
        cur_image[temp_image > truncate] = 1
        cur_image = np.transpose(cur_image, (2, 0, 1)).astype(np.int8)
        
        np.save(file = os.path.join(save_folder, f"size{size}", "images", f"img_{i:06d}"), arr = cur_image)
        
    return
    

In [ ]:
load_path = '../../../data/tonsil'
df = df_clean
xrange = [ 4, 5, 6, 7, 8]
yrange = [8, 9, 10, 11, 12, 13]
ratio = np.sqrt((53500/63)/(313338/99))

save_folder = os.path.join(load_path, "processed_data", "single_cell_images")
size = 512
folder = os.path.join(save_folder, f"size{size}", "images")
if not os.path.exists(folder):
    os.makedirs(folder)

# truncate = 4000 is ~ quantile 0.9 in this dataset
save_train_test_data(df, size, load_path, save_folder, xrange, yrange, ratio,truncate = 4000, shape_of_each_view=(1008, 1344),
                              channels = ('CD45', 'HOECHST1'), shape_of_views=(11, 9), pad = 500, verbose = True)